# NNFFLIB Inference Tutorial

In this tutorial, we will use a pretrained NNFFLIB model to compute certain properties and perform MD simulations. We will start from a pretrained model of ethanal. The model is stored at the location `models/ethanal`. That model can be loaded as follows:

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf

In [ ]:
from nnfflib.l1mlp import L1MLP

In [ ]:
model = L1MLP.from_restore_file('models/ethanal', float_type = 64, xla = False)

One can ignore the warnings about checkpointed values that are not bound. The arguments to load the models were explained in the notebook to train an MLP. Here, it is important to stress that one can change the precision of the model, i.e. using float32 or float64. If one wants to use NNFFLIB to perform geometry optimizations, float64 is required. Otherwise, one can use float32 without any issues.

## Validating xyz files

As a first example, we will use NNFFLIB to compute energies and forces for a single configuration. First, we will load the first configuration from the validation set:

In [ ]:
from nnfflib.help_functions import load_xyz
for data in load_xyz('data/validation.xyz'):
    energy = data['energy']
    numbers = data['Z']
    positions = data['pos']
    forces = data['force']
    break

Next, we will compute the MLP predictions

In [ ]:
output = model.compute(positions, numbers, rvec = np.eye(3) * 100, list_of_properties = ['energy', 'forces'])
print(output)

By specifying the `list_of_properties` argument, one can choose what kind of properties that NNFFLIB will compute. If stresses are required, one should add `'stress'` to compute the stresses in GPa. For this specific configuration the energy and force MAE errors are:

In [ ]:
def MAE(x, y):
    return np.mean(np.abs(x - y))
print('MAE energy: ', MAE(energy, output['energy']), ' eV')
print('MAE forces: ', MAE(forces, output['forces']), ' eV/A')

## Molecular dynamics simulation

Molecular dynamics simulations with the NNFFLIB can be done with [Yaff](https://github.com/molmod/yaff). Basically, NNFFLIB provides a Yaff wrapper around the `model.compute()` function (see above). It is also possible to perform MD simulations with other codes after writing your own wrapper around that function. Here, in Yaff, an NVT simulation can be performed as follows:

In [ ]:
from yaff import *
from nnfflib.md import NVT
system = System(numbers, positions * angstrom)
NVT(system, model, steps = 1000, screenprint = 100, nprint = 25, dt = 0.5, temp = 300, name = 'md_run1')

The code above should have performed an NVT simulation for `steps=1000` steps, with a timestep of `dt` fs at a temperature of `temp`. It will write the positions and centers every `nprint` steps in an extended xyz file at the location `name`. Every `screenprint` steps, progress will be written to the screen.

## Computation of the Hessian matrix

In this section, we will compute the Hessian matrix with NNFFLIB, which can be used for further processing. First, we have to compute the equilibrium geometry. This should take approximately 30 optimization steps. Only after the optimization, the Hessian will be positive definite.

In [ ]:
from nnfflib.md import Optimize
positions, rvec = Optimize(model, positions, numbers, rvec=np.eye(3)*100, log='opt.xyz', fullcell=False)

With the `fullcell` argument one can specify whether or not also optimize the energy with respect to the full cell matrix. Next, one should compute the full extended Hessian (this should take a few seconds on a GPU):

In [ ]:
full_hessian = model.compute_hessian(positions, numbers, rvec = np.eye(3) * 100, include_rvec = False)
print(full_hessian.shape)

For a system with N atoms, the Hessian matrix has a shape of 3N by 3N. If `include_rvec` is `True`, an extended Hessian is computed with the shape 3N+9 by 3N+9 since all 9 cell degrees of freedom are included.

In [ ]:
print(np.linalg.eigh(full_hessian)[0])

As a check, we have computed all the eigenvalues of the Hessian. Only the first 6 are (almost) zero, which correspond to the translational and rotational degrees of freedom in the system.